## Model Training

#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [1]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.9 MB/s eta 0:00:00


In [2]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

#### Import the CSV Data as Pandas DataFrame

In [3]:
df = pd.read_csv('data_UCP.csv')

#### Show Top 5 Records

In [4]:
df.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage (kmpl),Engine (CC),Power (bhp),Seats,Price
0,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50
1,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,4.50
2,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00
3,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74
4,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,3.50


#### Preparing X and Y variables

In [5]:
X = df.drop(columns=["Price"],axis=1)

In [6]:
X.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage (kmpl),Engine (CC),Power (bhp),Seats
0,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0
1,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0
2,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0
3,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0
4,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0


In [7]:
print(f"Categories in 'Location' variable: {df['Location'].unique()}")

print(f"Categories in 'Fuel_Type' variable:  {df['Fuel_Type'].unique()}")

print(f"Categories in 'Transmission' variable: {df['Transmission'].unique()}")

print(f"Categories in 'Owner_Type' variable: {df['Owner_Type'].unique()}")

Categories in 'Location' variable: ['Pune' 'Chennai' 'Coimbatore' 'Jaipur' 'Mumbai' 'Kochi' 'Kolkata' 'Delhi'
 'Bangalore' 'Hyderabad' 'Ahmedabad']
Categories in 'Fuel_Type' variable:  ['Diesel' 'Petrol']
Categories in 'Transmission' variable: ['Manual' 'Automatic']
Categories in 'Owner_Type' variable: ['First' 'Second' 'Fourth & Above' 'Third']


In [8]:
y = df['Price']

In [9]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [10]:
X = preprocessor.fit_transform(X)

In [11]:
X.shape

(5843, 25)

In [12]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((4674, 25), (1169, 25))

#### Create an Evaluate Function to give all metrics after model Training

In [13]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [14]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 6.3024
- Mean Absolute Error: 3.8117
- R2 Score: 0.6948
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 8.1720
- Mean Absolute Error: 3.7397
- R2 Score: 0.4210


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.8673
- Mean Absolute Error: 3.8365
- R2 Score: 0.6377
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.0509
- Mean Absolute Error: 3.5429
- R2 Score: 0.6825


Ridge
Model performance for Training set
- Root Mean Squared Error: 6.2998
- Mean Absolute Error: 3.7968
- R2 Score: 0.6951
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 8.0789
- Mean Absolute Error: 3.7226
- R2 Score: 0.4341


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 3.9895
- Mean Absolute Error: 1.7717
- R2 Score: 0.8777
-----------------------

### Results

In [15]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
7,CatBoosting Regressor,0.925263
5,Random Forest Regressor,0.917970
6,XGBRegressor,0.891640
4,Decision Tree,0.840865
3,K-Neighbors Regressor,0.814425
8,AdaBoost Regressor,0.692038
1,Lasso,0.682537
2,Ridge,0.434083
0,Linear Regression,0.420964
